# Data Preparation 2

- Split date feature
- API values imputation for missing values
- Drop outliers
- Label encoding

#### Import Libraries

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
%matplotlib inline

#### Read Data

In [2]:
df = pd.read_csv('../../../../datasets/parte1/weatherAUS.csv')
df_api = pd.read_csv('../../../../datasets/parte1/api.csv')

#### Handle Missing Values

##### Merge API data by Date and Location

In [3]:
# Merge API data based on Location and Date using left merge
merged_df = pd.merge(df, df_api, on=['Location', 'Date'], how='left', suffixes=('', '_df2'))

for col in df.columns:
    if col in df_api and col not in ["Location", "Date"]:
        merged_df[col].fillna(merged_df[col + '_df2'], inplace=True)

# Drop the columns ending with '_df2'
merged_df.drop(columns=merged_df.filter(like='_df2').columns, inplace=True)

df = merged_df

##### Drop other columns missing values

In [4]:
df.dropna(inplace=True)

#### Drop Outliers

In [5]:
for feature in df.select_dtypes(include=['int', 'float']).columns:
    Q1 = df[feature].quantile(0.25)
    Q3 = df[feature].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Remove outliers for the current numeric feature
    df = df[(df[feature] >= lower_bound) & (df[feature] <= upper_bound)]

#### Split Date Feature

Instead of using the full date, extracting just the month is much more valuable due to rain seasonality

In [6]:
df['Date'] = pd.to_datetime(df['Date'], format="%Y-%m-%d", utc=True)
df['Month'] = df['Date'].dt.month
df.drop(['Date'], inplace=True, axis=1)

#### Label Encoding

Converts categorical data into numerical format

In [7]:
label_encoder = LabelEncoder()

df['Location'] = label_encoder.fit_transform(df['Location'])
df['WindGustDir'] = label_encoder.fit_transform(df['WindGustDir'])
df['WindDir9am'] = label_encoder.fit_transform(df['WindDir9am'])
df['WindDir3pm'] = label_encoder.fit_transform(df['WindDir3pm'])
df['RainToday'] = label_encoder.fit_transform(df['RainToday'])
df['RainTomorrow'] = label_encoder.fit_transform(df['RainTomorrow'])

#### Write Prepared Data

In [8]:
df.to_csv('../../../../datasets/parte1/dataset_cleaned.csv', index=False)